# 为reaction添加seed、bigg、kegg linker和ec-code

In [20]:
import pandas as pd
import re

# 读取数据
xlsx_file = '4. 添加linker/14067_rxn_3.xlsx'  # 这里替换为你的xlsx文件路径
tsv_file = '4. 添加linker/modelseed_reactions.tsv'  # 这里替换为你的tsv文件路径
# 读取 Excel 和 TSV 文件
df_xlsx = pd.read_excel(xlsx_file)
df_tsv = pd.read_csv(tsv_file, sep='\t')

# 添加 seed.reaction 列：从 ID 列提取 rxn{5}
df_xlsx['seed.reaction'] = df_xlsx['ID'].str.extract(r'(rxn\d{5})')

# 函数：从 aliases 列提取 BiGG 和 KEGG 信息（保留多个）
def extract_bigg_kegg(aliases):
    # 确保 aliases 是字符串类型
    if isinstance(aliases, str):
        # 提取所有 BiGG 名称
        bigg_matches = re.findall(r'BiGG: ([^|]+)', aliases)
        bigg = '; '.join(bigg_matches) if bigg_matches else ''
        
        # 提取所有 KEGG 编号
        kegg_matches = re.findall(r'KEGG: ([^|]+)', aliases)
        kegg = '; '.join(kegg_matches) if kegg_matches else ''
        
        return bigg, kegg
    return '', ''  # 如果 aliases 不是字符串，返回空值

# 函数：获取 ec_numbers，只保留第一个 EC 编号
def extract_ec_numbers(ec_numbers):
    if pd.notnull(ec_numbers):
        # 如果包含多个 EC 编号，则只保留第一个（以 "|" 分隔）
        ec_list = ec_numbers.split('|')
        return ec_list[0]
    return ''  # 如果为空，则返回空字符串

# 创建 BiGG.reaction, KEGG.reaction, 和 ec-code 列
df_xlsx['BiGG.reaction'] = ''
df_xlsx['KEGG.reaction'] = ''
df_xlsx['ec-code'] = ''

# 遍历 Excel 文件中的每一行，进行匹配并更新列
for idx, row in df_xlsx.iterrows():
    seed_reaction = row['seed.reaction']
    
    # 根据 seed.reaction 去匹配 tsv 文件
    matched_row = df_tsv[df_tsv['id'] == seed_reaction]
    if not matched_row.empty:
        # 提取 aliases 和 ec_numbers
        aliases = matched_row.iloc[0]['aliases']
        ec_numbers = matched_row.iloc[0]['ec_numbers']
        
        # 提取 BiGG 和 KEGG 信息
        bigg, kegg = extract_bigg_kegg(aliases)
        
        # 更新 xlsx 数据框
        df_xlsx.at[idx, 'BiGG.reaction'] = bigg
        df_xlsx.at[idx, 'KEGG.reaction'] = kegg
        df_xlsx.at[idx, 'ec-code'] = extract_ec_numbers(ec_numbers)

# 保存修改后的文件
df_xlsx.to_excel('14067_rxn_linker1.xlsx', index=False)

print("操作完成！")


操作完成！


# 根据bigg文件去用rxn搜索再次获取相应biggid

In [21]:
import pandas as pd
import re

def process_files(xlsx_path, tsv_path, output_path):
    """
    处理xlsx和tsv文件，根据匹配规则更新BiGG.reaction列
    支持多个bigg_id的情况，用分号分隔
    
    Parameters:
    xlsx_path (str): 输入xlsx文件路径
    tsv_path (str): 输入tsv文件路径
    output_path (str): 输出xlsx文件路径
    """
    # 读取xlsx文件
    xlsx_df = pd.read_excel(xlsx_path)
    
    # 读取tsv文件
    tsv_df = pd.read_csv(tsv_path, sep='\t')
    
    # 获取需要查找的rxn模式
    rxn_pattern = re.compile(r'rxn\d{5}_c0')
    rxn_ids = []
    
    # 从xlsx文件中提取符合模式的ID
    for idx, row in xlsx_df.iterrows():
        if pd.isna(row['BiGG.reaction']):  # 检查BiGG.reaction是否为空
            match = rxn_pattern.search(str(row['ID']))
            if match:
                rxn_ids.append((idx, match.group(0)[:8]))  # 只取rxn部分，不含_c0
    
    # 在tsv文件中查找对应的bigg_id
    for idx, rxn_id in rxn_ids:
        # 构造搜索模式
        search_pattern = f"SEED Reaction: http://identifiers.org/seed.reaction/{rxn_id}"
        
        # 在database_links列中查找所有匹配项
        matches = tsv_df[tsv_df['database_links'].str.contains(search_pattern, na=False)]
        
        if not matches.empty:
            # 获取所有匹配的bigg_id并用分号连接
            bigg_ids = matches['bigg_id'].unique()  # 使用unique()去除重复值
            bigg_ids_str = '; '.join(bigg_ids)
            
            # 更新到xlsx文件中
            xlsx_df.at[idx, 'BiGG.reaction'] = bigg_ids_str
            
            # 打印匹配信息（可选，用于调试）
            print(f"Found matches for {rxn_id}: {bigg_ids_str}")
    
    # 保存更新后的xlsx文件
    xlsx_df.to_excel(output_path, index=False)
    print(f"处理完成，结果已保存到: {output_path}")
    
    # 打印统计信息
    total_processed = len(rxn_ids)
    total_matched = len(xlsx_df[xlsx_df['BiGG.reaction'].notna()])
    print(f"\n统计信息:")
    print(f"处理的rxn数量: {total_processed}")
    print(f"成功匹配的数量: {total_matched}")

# 使用示例
if __name__ == "__main__":
    # 替换为实际的文件路径
    xlsx_file = "4. 添加linker/14067_rxn_linker1.xlsx"
    tsv_file = "4. 添加linker/bigg_models_reactions.tsv"
    output_file = "4. 添加linker/14067_rxn_linker2.xlsx"
    
    process_files(xlsx_file, tsv_file, output_file)

Found matches for rxn00011: PDHam1hi; PDHam1mi
Found matches for rxn00048: RBFSb; RIBFS
Found matches for rxn00113: CPS
Found matches for rxn00264: MOX
Found matches for rxn00302: r0120
Found matches for rxn00304: GTPOPm; PYK3
Found matches for rxn00345: NACASPAH
Found matches for rxn00360: BPNT2
Found matches for rxn00411: PYK4
Found matches for rxn00426: SERHL
Found matches for rxn00441: r0163
Found matches for rxn00460: PYK2
Found matches for rxn00469: ACODA; ACODA_1
Found matches for rxn00471: ORNCD
Found matches for rxn00484: LTDCL
Found matches for rxn00490: AROH
Found matches for rxn00517: PYK5
Found matches for rxn00529: TYRCBOX
Found matches for rxn00627: KHK
Found matches for rxn00657: APAT2rm
Found matches for rxn00780: TRIOK
Found matches for rxn00801: FUMAH
Found matches for rxn00802: ARGSL
Found matches for rxn00840: DAPOP; r0280
Found matches for rxn00905: 2DHPFALDL
Found matches for rxn00935: DMALRED
Found matches for rxn00999: PPOR
Found matches for rxn01014: HPYRDC; H

# 针对手动添加的交换反应，添加bigg id

可逆的交换反应需要手动更新

In [27]:
import pandas as pd
import re

def extract_metabolite(definition):
    """从Definition列提取代谢物名称并转换为BiGG reaction ID格式"""
    if pd.isna(definition):
        return None
    # 匹配方程式左边的代谢物名称
    match = re.match(r'(\w+)_e0\[e0\]\s*=>', definition.strip())
    if match:
        metabolite = match.group(1)
        return f"EX_{metabolite}_e"
    return None

def extract_seed_reaction(database_links):
    """提取第一个SEED reaction ID"""
    if pd.isna(database_links):
        return None
    
    # 匹配SEED Reaction ID
    seed_pattern = r'SEED Reaction: http://identifiers\.org/seed\.reaction/(rxn\d+)'
    match = re.search(seed_pattern, database_links)
    
    return match.group(1) if match else None

def modify_equation(definition):
    """修改方程式格式"""
    if pd.isna(definition):
        return None
    
    # 添加(1)前缀并保持其他格式不变
    return f"(1) {definition}"

def process_files(xlsx_path, tsv_path, output_path):
    """
    处理xlsx和tsv文件，转换反应方程式并提取相关ID
    
    Parameters:
    xlsx_path (str): 输入xlsx文件路径
    tsv_path (str): 输入tsv文件路径
    output_path (str): 输出xlsx文件路径
    """
    # 读取文件
    xlsx_df = pd.read_excel(xlsx_path)
    tsv_df = pd.read_csv(tsv_path, sep='\t')
    
    # 如果seed.reaction列不存在，创建它
    if 'seed.reaction' not in xlsx_df.columns:
        xlsx_df['seed.reaction'] = None
    
    # 处理每一行
    for idx, row in xlsx_df.iterrows():
        if row['source'] == 'exchange reaction':
            # 提取代谢物并创建BiGG reaction ID
            bigg_id = extract_metabolite(row['Definition'])
            
            if bigg_id:
                # 更新BiGG.reaction列
                if pd.isna(row['BiGG.reaction']):
                    xlsx_df.at[idx, 'BiGG.reaction'] = bigg_id
                elif bigg_id not in str(row['BiGG.reaction']):
                    # 如果已经有值，则添加到现有值后面
                    xlsx_df.at[idx, 'BiGG.reaction'] = f"{row['BiGG.reaction']}; {bigg_id}"
                
                # 在tsv文件中查找匹配项
                matches = tsv_df[tsv_df['bigg_id'] == bigg_id]
                if not matches.empty:
                    # 获取database_links并提取SEED reaction ID
                    database_links = matches.iloc[0]['database_links']
                    seed_id = extract_seed_reaction(database_links)
                    if seed_id:
                        xlsx_df.at[idx, 'seed.reaction'] = seed_id
            
            # 修改方程式格式
            xlsx_df.at[idx, 'Definition'] = modify_equation(row['Definition'])
    
    # 保存结果
    xlsx_df.to_excel(output_path, index=False)
    print(f"处理完成，结果已保存到: {output_path}")
    
    # 打印统计信息
    exchange_reactions = xlsx_df['source'] == 'exchange reaction'
    total_exchange = exchange_reactions.sum()
    processed = xlsx_df[exchange_reactions]['BiGG.reaction'].notna().sum()
    seed_matched = xlsx_df[exchange_reactions]['seed.reaction'].notna().sum()
    
    print(f"\n统计信息:")
    print(f"exchange reaction总数: {total_exchange}")
    print(f"成功处理的反应数: {processed}")
    print(f"成功匹配SEED reaction ID数: {seed_matched}")

# 使用示例
if __name__ == "__main__":
    # 替换为实际的文件路径
    xlsx_file = "4. 添加linker/14067_rxn_linker2.xlsx"
    tsv_file = "4. 添加linker/bigg_models_reactions.tsv"
    output_file = "4. 添加linker/14067_rxn_linker3.xlsx"
    
    process_files(xlsx_file, tsv_file, output_file)

处理完成，结果已保存到: 14067_rxn_linker3.xlsx

统计信息:
exchange reaction总数: 161
成功处理的反应数: 140
成功匹配SEED reaction ID数: 136


# 根据bigg去获取 biocyc rhea metanetx linker

In [28]:
import pandas as pd
import re

def extract_last_rhea(database_links):
    """提取最后一个RHEA ID"""
    if pd.isna(database_links):
        return None
    
    rhea_pattern = r'RHEA: http://identifiers\.org/rhea/(\d+)'
    matches = re.finditer(rhea_pattern, database_links)
    
    # 转换迭代器到列表并获取最后一个匹配
    matches_list = list(matches)
    if matches_list:
        return matches_list[-1].group(1)
    return None

def extract_metanetx(database_links):
    """提取MetaNetX reaction ID"""
    if pd.isna(database_links):
        return None
    
    metanetx_pattern = r'MetaNetX \(MNX\) Equation: http://identifiers\.org/metanetx\.reaction/(MNXR\d+)'
    match = re.search(metanetx_pattern, database_links)
    
    return match.group(1) if match else None

def extract_biocyc(database_links):
    """提取BioCyc ID"""
    if pd.isna(database_links):
        return None
    
    biocyc_pattern = r'BioCyc: http://identifiers\.org/biocyc/(META:[^;]+)'
    match = re.search(biocyc_pattern, database_links)
    
    return match.group(1) if match else None

def process_files(xlsx_path, tsv_path, output_path):
    """
    处理xlsx和tsv文件，提取相关ID并创建新列
    
    Parameters:
    xlsx_path (str): 输入xlsx文件路径
    tsv_path (str): 输入tsv文件路径
    output_path (str): 输出xlsx文件路径
    """
    # 读取文件
    xlsx_df = pd.read_excel(xlsx_path)
    tsv_df = pd.read_csv(tsv_path, sep='\t')
    
    # 初始化新列
    xlsx_df['rhea'] = None
    xlsx_df['MetanetX.reaction'] = None
    xlsx_df['biocyc'] = None
    
    # 处理每一行
    for idx, row in xlsx_df.iterrows():
        bigg_reaction = row['BiGG.reaction']
        
        # 只处理非空的BiGG.reaction
        if pd.notna(bigg_reaction):
            # 在tsv文件中查找匹配的行
            matches = tsv_df[tsv_df['bigg_id'] == bigg_reaction]
            
            if not matches.empty:
                # 获取第一个匹配行的database_links
                database_links = matches.iloc[0]['database_links']
                
                # 提取各种ID
                rhea_id = extract_last_rhea(database_links)
                metanetx_id = extract_metanetx(database_links)
                biocyc_id = extract_biocyc(database_links)
                
                # 更新DataFrame
                xlsx_df.at[idx, 'rhea'] = rhea_id
                xlsx_df.at[idx, 'MetanetX.reaction'] = metanetx_id
                xlsx_df.at[idx, 'biocyc'] = biocyc_id
    
    # 保存结果
    xlsx_df.to_excel(output_path, index=False)
    print(f"处理完成，结果已保存到: {output_path}")
    
    # 打印统计信息
    total_processed = xlsx_df['BiGG.reaction'].notna().sum()
    rhea_matched = xlsx_df['rhea'].notna().sum()
    metanetx_matched = xlsx_df['MetanetX.reaction'].notna().sum()
    biocyc_matched = xlsx_df['biocyc'].notna().sum()
    
    print(f"\n统计信息:")
    print(f"处理的BiGG.reaction数量: {total_processed}")
    print(f"成功匹配RHEA ID数量: {rhea_matched}")
    print(f"成功匹配MetaNetX ID数量: {metanetx_matched}")
    print(f"成功匹配BioCyc ID数量: {biocyc_matched}")

# 使用示例
if __name__ == "__main__":
    # 替换为实际的文件路径
    xlsx_file = "4. 添加linker/14067_rxn_linker3.xlsx"
    tsv_file = "4. 添加linker/bigg_models_reactions.tsv"
    output_file = "4. 添加linker/14067_rxn_linker4.xlsx"
    
    process_files(xlsx_file, tsv_file, output_file)

处理完成，结果已保存到: 14067_rxn_linker4.xlsx

统计信息:
处理的BiGG.reaction数量: 1111
成功匹配RHEA ID数量: 633
成功匹配MetaNetX ID数量: 1029
成功匹配BioCyc ID数量: 610


# 提取代谢物

In [3]:
import pandas as pd
import re

def extract_metabolites(equation):
    """从反应方程式中提取所有代谢物（包括compartment标记）"""
    if pd.isna(equation):
        return []
    
    # 更新的正则表达式模式，匹配:
    # 1. 带系数的代谢物: (数字) 代谢物[compartment]
    # 2. 不带系数的代谢物: 代谢物[compartment]
    pattern = r'(?:\([\d.]+\)\s*)?([\w\-]+)\[([ce]\d+)\]'
    
    # 找出所有匹配项
    matches = re.finditer(pattern, equation)
    
    # 提取代谢物名称和compartment
    metabolites = [f"{match.group(1)}[{match.group(2)}]" for match in matches]
    
    return metabolites

def process_file(input_path, output_path):
    """
    处理Excel文件，提取并去重代谢物
    
    Parameters:
    input_path (str): 输入xlsx文件路径
    output_path (str): 输出xlsx文件路径
    """
    # 读取Excel文件
    df = pd.read_excel(input_path)
    
    # 存储所有唯一的代谢物
    all_metabolites = set()
    
    # 从每个方程式中提取代谢物
    for equation in df['Equation']:
        metabolites = extract_metabolites(equation)
        all_metabolites.update(metabolites)
    
    # 转换为排序后的列表
    sorted_metabolites = sorted(list(all_metabolites))
    
    # 创建新的DataFrame
    output_df = pd.DataFrame({'metabolites': sorted_metabolites})
    
    # 保存到新的Excel文件
    output_df.to_excel(output_path, index=False)
    
    # 打印统计信息
    print(f"处理完成，结果已保存到: {output_path}")
    print(f"找到的唯一代谢物数量: {len(sorted_metabolites)}")
    
    # 打印一些示例进行验证
    print("\n代谢物示例:")
    # 打印部分带[c0]的代谢物
    c0_mets = [m for m in sorted_metabolites if '[c0]' in m][:3]
    print("带[c0]的代谢物示例:", c0_mets)
    
    # 打印部分带[e0]的代谢物
    e0_mets = [m for m in sorted_metabolites if '[e0]' in m][:3]
    print("带[e0]的代谢物示例:", e0_mets)

# 使用示例
if __name__ == "__main__":
    input_file = "14067gem20_CGXII.xlsx"
    output_file = "4. 添加linker/14067_met_linker1.xlsx"
    
    process_file(input_file, output_file)

处理完成，结果已保存到: 4. 添加linker/14067_met_linker1.xlsx
找到的唯一代谢物数量: 0

代谢物示例:
带[c0]的代谢物示例: []
带[e0]的代谢物示例: []


# 获取代谢物相关信息

In [1]:
import pandas as pd
import re

# 读取xlsx文件
df_xlsx = pd.read_excel('4. 添加linker/14067_met_linker1.xlsx')

# 读取tsv文件
df_tsv = pd.read_csv('4. 添加linker/modelseed_compounds.tsv', sep='\t', low_memory=False)

# 定义一个函数,用于匹配metabolite ID并从TSV中获取相应的数据
def get_metabolite_data(metabolite_id, name):
    if pd.notnull(name):
        return pd.Series(['']*11, index=['name', 'abbreviation', 'formula', 'mass', 'inchikey', 'charge', 'deltag', 'deltagerr', 'pka', 'pkb', 'smiles'])
    
    stripped_id = re.sub(r'\[[ce]0\]$', '', str(metabolite_id))
    matched_compound = df_tsv[df_tsv['id'] == stripped_id]
    
    if not matched_compound.empty:
        return matched_compound[['name', 'abbreviation', 'formula', 'mass', 'inchikey', 'charge', 'deltag', 'deltagerr', 'pka', 'pkb', 'smiles']].iloc[0]
    
    return pd.Series(['']*11, index=['name', 'abbreviation', 'formula', 'mass', 'inchikey', 'charge', 'deltag', 'deltagerr', 'pka', 'pkb', 'smiles'])

# 处理xlsx数据并添加匹配的tsv数据
metabolite_data = df_xlsx.apply(lambda x: get_metabolite_data(x['metabolites_id'], x['name']), axis=1)

df_xlsx['name'] = df_xlsx['name'].where(pd.notnull(df_xlsx['name']), metabolite_data['name'])
df_xlsx['abbreviation'] = df_xlsx['abbreviation'].where(pd.notnull(df_xlsx['abbreviation']), metabolite_data['abbreviation'])
df_xlsx['formula'] = df_xlsx['formula'].where(pd.notnull(df_xlsx['formula']), metabolite_data['formula'])
df_xlsx['mass'] = df_xlsx['mass'].where(pd.notnull(df_xlsx['mass']), metabolite_data['mass'])
df_xlsx['charge'] = df_xlsx['charge'].where(pd.notnull(df_xlsx['charge']), metabolite_data['charge'])
df_xlsx['pka'] = df_xlsx['pka'].where(pd.notnull(df_xlsx['pka']), metabolite_data['pka'])
df_xlsx['pkb'] = df_xlsx['pkb'].where(pd.notnull(df_xlsx['pkb']), metabolite_data['pkb'])
df_xlsx['smiles'] = df_xlsx['smiles'].where(pd.notnull(df_xlsx['smiles']), metabolite_data['smiles'])

df_xlsx['inchikey'] = metabolite_data['inchikey']
df_xlsx['deltaG'] = metabolite_data.apply(lambda x: f"{x['deltag']}±{x['deltagerr']} (kcal/mol)" if pd.notnull(x['deltag']) else '', axis=1)

# 将更新后的数据写入新的xlsx文件
df_xlsx.to_excel('14067_met_linker2.xlsx', index=False)
print('14067_met_linker2.xlsx 生成成功!')

14067_met_linker2.xlsx 生成成功!


# 获取代谢物相关linker

In [2]:
import pandas as pd
import re

# 读取Excel文件
df = pd.read_excel('14067_met_linker2.xlsx')

# 读取TSV文件，设置low_memory=False来避免混合类型警告
modelseed_df = pd.read_csv('4. 添加linker/modelseed_compounds.tsv', sep='\t', low_memory=False)

# 创建新列seed.compou，提取cpd编号
df['seed.compound'] = df['metabolites_id'].str.extract(r'(cpd\d+)')

# 创建新列用于存储BiGG和KEGG的标识符
df['BiGG.compound'] = ''
df['KEGG.compound'] = ''

# 遍历每一行
for index, row in df.iterrows():
    # 获取cpd编号
    cpd = row['seed.compound']
    if pd.notna(cpd):
        # 在modelseed_compounds中查找匹配的行
        match = modelseed_df[modelseed_df['id'] == cpd]
        
        if not match.empty:
            aliases = match.iloc[0]['aliases']
            
            # 检查aliases是否为有效字符串
            if isinstance(aliases, str):
                # 提取BiGG标识符
                bigg_match = re.search(r'BiGG: ([^|]+)', aliases)
                if bigg_match:
                    df.at[index, 'BiGG.compound'] = bigg_match.group(1)
                
                # 提取KEGG标识符
                kegg_match = re.search(r'KEGG: ([^|]+)', aliases)
                if kegg_match:
                    df.at[index, 'KEGG.compound'] = kegg_match.group(1)

# 保存结果到新的Excel文件
df.to_excel('14067_met_linker3.xlsx', index=False)

# 根据biggid去获取metabolite的MNX、Biocyc linker

In [3]:
import pandas as pd
import re

# 读取Excel文件和TSV文件
df = pd.read_excel('14067_met_linker3.xlsx')
bigg_df = pd.read_csv('4. 添加linker/bigg_models_metabolites.tsv', sep='\t', low_memory=False)

# 创建新列
df['MetaNetX.compound'] = ''
df['Biocyc'] = ''

# 遍历每一行
for index, row in df.iterrows():
    bigg_id = row['BiGG.compound']
    
    # 只处理BiGG.compound非空的行
    if pd.notna(bigg_id):
        # 在bigg_models_metabolites中查找匹配的行
        match = bigg_df[bigg_df['universal_bigg_id'] == bigg_id]
        
        if not match.empty:
            database_links = match.iloc[0]['database_links']
            
            # 确保database_links是字符串
            if isinstance(database_links, str):
                # 提取MetaNetX ID
                metanetx_match = re.search(r'MetaNetX \(MNX\) Chemical: http://identifiers\.org/metanetx\.chemical/([^;]+)', database_links)
                if metanetx_match:
                    df.at[index, 'MetaNetX.compound'] = metanetx_match.group(1).strip()
                
                # 提取BioCyc ID
                biocyc_match = re.search(r'BioCyc: http://identifiers\.org/biocyc/([^;]+)', database_links)
                if biocyc_match:
                    df.at[index, 'Biocyc'] = biocyc_match.group(1).strip()

# 保存结果到新的Excel文件
df.to_excel('14067_met_linker4.xlsx', index=False)

# 进一步添加CHIBI等信息

In [1]:
import pandas as pd
import re
import os

def main():
    # 读取文件
    xlsx_file = 'met.xlsx'
    tsv_file = '4. 添加linker/bigg_models_metabolites.tsv'
    
    # 验证文件是否存在
    if not os.path.exists(xlsx_file):
        print(f"错误: 文件不存在: {xlsx_file}")
        return
    if not os.path.exists(tsv_file):
        print(f"错误: 文件不存在: {tsv_file}")
        return
    
    # 读取文件内容
    try:
        xlsx_df = pd.read_excel(xlsx_file)
        tsv_df = pd.read_csv(tsv_file, sep='\t')
    except Exception as e:
        print(f"读取文件时出错: {e}")
        return
    
    # 检查必要的列是否存在
    if 'BiGG.compound' not in xlsx_df.columns:
        print("错误: Excel文件中未找到'BiGG.compound'列")
        print(f"可用列: {xlsx_df.columns.tolist()}")
        return
    
    if 'universal_bigg_id' not in tsv_df.columns or 'database_links' not in tsv_df.columns:
        print("错误: TSV文件中缺少必要的列")
        print(f"可用列: {tsv_df.columns.tolist()}")
        return
    
    # 创建新列
    xlsx_df['Reactome Compound'] = None
    xlsx_df['CHEBI'] = None
    xlsx_df['Human Metabolome Database'] = None
    
    # 定义正则表达式模式，用于提取ID
    reactome_pattern = r'Reactome Compound: http://identifiers.org/reactome/(R-ALL-[0-9]+)'
    chebi_pattern = r'CHEBI: http://identifiers.org/chebi/CHEBI:([0-9]+)'
    hmdb_pattern = r'Human Metabolome Database: http://identifiers.org/hmdb/(HMDB[0-9]+)'
    
    # 创建查找字典，加快匹配速度
    bigg_to_links = {}
    for _, row in tsv_df.iterrows():
        if pd.notna(row['universal_bigg_id']) and pd.notna(row.get('database_links', '')):
            bigg_to_links[row['universal_bigg_id']] = row['database_links']
    
    # 处理每一行
    matches_found = 0
    total_rows = len(xlsx_df)
    
    print(f"正在处理 {total_rows} 行数据...")
    
    for idx, row in xlsx_df.iterrows():
        bigg_compound = row.get('BiGG.compound', '')
        
        # 跳过空值
        if pd.isna(bigg_compound) or bigg_compound == '':
            continue
        
        # 查找匹配
        if bigg_compound in bigg_to_links:
            database_links = bigg_to_links[bigg_compound]
            matches_found += 1
            
            # 提取ID
            reactome_matches = re.findall(reactome_pattern, database_links)
            chebi_matches = re.findall(chebi_pattern, database_links)
            hmdb_matches = re.findall(hmdb_pattern, database_links)
            
            # 赋值第一个匹配项，如果没有则保持为None
            xlsx_df.at[idx, 'Reactome Compound'] = reactome_matches[0] if reactome_matches else None
            xlsx_df.at[idx, 'CHEBI'] = chebi_matches[0] if chebi_matches else None
            xlsx_df.at[idx, 'Human Metabolome Database'] = hmdb_matches[0] if hmdb_matches else None
    
    # 生成输出文件名
    output_file = os.path.splitext(xlsx_file)[0] + "_updated.xlsx"
    
    # 保存结果
    try:
        xlsx_df.to_excel(output_file, index=False)
        print(f"处理完成。已将结果保存为: '{output_file}'")
        print(f"总行数: {total_rows}")
        print(f"找到匹配项: {matches_found}")
    except Exception as e:
        print(f"保存文件时出错: {e}")

if __name__ == "__main__":
    main()

正在处理 1652 行数据...
处理完成。已将结果保存为: 'met_updated.xlsx'
总行数: 1652
找到匹配项: 1188


In [11]:
import pandas as pd
import time
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def extract_pubchem_id(href_or_text):
    """
    从PubChem链接中提取Substance ID
    例如从 "http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?sid=8145132" 提取 "8145132"
    """
    # 尝试从URL中提取sid参数
    sid_match = re.search(r'sid=(\d+)', href_or_text, re.IGNORECASE)
    if sid_match:
        return sid_match.group(1)
    
    # 尝试从文本中提取数字（如果没有sid参数但是有数字）
    number_match = re.search(r'(\d+)', href_or_text)
    if number_match:
        return number_match.group(1)
    
    # 如果无法提取，返回原始文本
    return href_or_text

def get_pubchem_id_from_reactome(driver, reactome_id):
    """
    从Reactome网页获取PubChem Substance ID，使用更灵活的查找方法
    
    Args:
        driver: WebDriver实例
        reactome_id: Reactome Compound ID
        
    Returns:
        PubChem Substance ID 或 None
    """
    try:
        # 构建URL
        url = f"https://reactome.org/content/detail/{reactome_id}#Homo%20sapiens"
        print(f"访问URL: {url}")
        driver.get(url)
        
        # 给JavaScript额外的时间运行
        time.sleep(3)
        
        # 方法1: 查找包含"PubChem Substance"文本的标签元素
        try:
            # 查找包含"PubChem Substance"文本的所有div元素
            substance_labels = driver.find_elements(By.XPATH, 
                "//div[contains(text(), 'PubChem Substance')]")
            
            if substance_labels:
                print(f"找到 {len(substance_labels)} 个PubChem Substance标签")
                
                for label in substance_labels:
                    # 找到包含此标签的fieldset-pair-container
                    try:
                        container = label.find_element(By.XPATH, 
                            "./ancestor::div[contains(@class, 'fieldset-pair-container')]")
                        
                        # 在容器中查找所有链接
                        links = container.find_elements(By.TAG_NAME, "a")
                        
                        for link in links:
                            href = link.get_attribute("href")
                            if href and "pubchem.ncbi.nlm.nih.gov/summary" in href:
                                pubchem_id = extract_pubchem_id(href)
                                print(f"方法1找到PubChem ID: {pubchem_id}")
                                return pubchem_id
                    except Exception as e:
                        print(f"处理标签时出错: {e}")
            else:
                print("未找到PubChem Substance标签")
        except Exception as e:
            print(f"方法1查找失败: {e}")
        
        # 方法2: 查找所有带有pubchem链接的元素
        try:
            pubchem_links = driver.find_elements(By.XPATH, 
                "//a[contains(@href, 'pubchem.ncbi.nlm.nih.gov/summary')]")
            
            if pubchem_links:
                print(f"找到 {len(pubchem_links)} 个PubChem链接")
                
                for link in pubchem_links:
                    href = link.get_attribute("href")
                    pubchem_id = extract_pubchem_id(href)
                    print(f"方法2找到PubChem ID: {pubchem_id}")
                    return pubchem_id
            else:
                print("未找到任何PubChem链接")
        except Exception as e:
            print(f"方法2查找失败: {e}")
            
        # 方法3: 查找所有fieldset元素，检查是否包含PubChem信息
        try:
            fieldsets = driver.find_elements(By.TAG_NAME, "fieldset")
            
            for fieldset in fieldsets:
                # 检查是否为Cross References部分
                legends = fieldset.find_elements(By.TAG_NAME, "legend")
                if legends and "Cross References" in legends[0].text:
                    # 在这个fieldset中查找包含PubChem的div
                    pubchem_divs = fieldset.find_elements(By.XPATH, 
                        ".//div[contains(text(), 'PubChem')]")
                    
                    if pubchem_divs:
                        # 找到每个PubChem标签附近的链接
                        for div in pubchem_divs:
                            # 查找与此PubChem标签相关的链接
                            container = div.find_element(By.XPATH, 
                                "./ancestor::div[contains(@class, 'fieldset-pair-container')]")
                            links = container.find_elements(By.TAG_NAME, "a")
                            
                            for link in links:
                                href = link.get_attribute("href")
                                if href and "pubchem" in href:
                                    pubchem_id = extract_pubchem_id(href)
                                    print(f"方法3找到PubChem ID: {pubchem_id}")
                                    return pubchem_id
            
            print("未在Cross References部分找到PubChem信息")
        except Exception as e:
            print(f"方法3查找失败: {e}")
                
    except Exception as e:
        print(f"处理 {reactome_id} 时出错: {e}")
    
    return None

def scrape_reactome_pubchem(input_file):
    """
    从Reactome网站获取PubChem Substance ID，先去重Reactome IDs
    
    Args:
        input_file (str): 包含Reactome Compound列的Excel文件路径
    """
    # 读取Excel文件
    print(f"正在读取Excel文件: {input_file}")
    try:
        df = pd.read_excel(input_file)
    except Exception as e:
        print(f"读取Excel文件时出错: {e}")
        return
    
    # 检查'Reactome Compound'列是否存在
    if 'Reactome Compound' not in df.columns:
        print("错误: Excel文件中未找到'Reactome Compound'列")
        print(f"可用列: {df.columns.tolist()}")
        return
    
    # 创建新列用于存储PubChem Substance ID
    df['PubChem Substance'] = None
    
    # 获取所有非空的Reactome Compound IDs
    non_empty_mask = df['Reactome Compound'].notna() & (df['Reactome Compound'].astype(str) != '')
    reactome_ids = df.loc[non_empty_mask, 'Reactome Compound'].tolist()
    
    # 去重Reactome IDs
    unique_reactome_ids = list(set(reactome_ids))
    
    print(f"总行数: {len(df)}")
    print(f"包含Reactome Compound ID的行数: {len(reactome_ids)}")
    print(f"唯一Reactome Compound ID数量: {len(unique_reactome_ids)}")
    
    if len(unique_reactome_ids) == 0:
        print("未找到Reactome Compound ID。程序退出。")
        return
    
    # 创建字典用于存储Reactome ID到PubChem ID的映射
    reactome_to_pubchem = {}
    
    # 初始化Edge浏览器
    print("\n正在初始化Edge WebDriver...")
    edge_options = Options()
    edge_options.add_argument("--headless")  # 无界面模式运行
    driver = webdriver.Edge(options=edge_options)
    
    try:
        # 处理每个唯一的Reactome Compound ID
        processed_count = 0
        success_count = 0
        
        for reactome_id in unique_reactome_ids:
            processed_count += 1
            print(f"\n正在处理唯一ID {processed_count}/{len(unique_reactome_ids)}: {reactome_id}")
            
            # 获取PubChem ID
            pubchem_id = get_pubchem_id_from_reactome(driver, reactome_id)
            
            # 保存结果到字典
            if pubchem_id:
                reactome_to_pubchem[reactome_id] = pubchem_id
                success_count += 1
            
            # 暂停以避免过度请求服务器
            time.sleep(2)
        
        # 将结果分配回原始数据框
        print("\n将结果分配回原始数据框...")
        
        for idx, row in df[non_empty_mask].iterrows():
            reactome_id = row['Reactome Compound']
            if reactome_id in reactome_to_pubchem:
                df.at[idx, 'PubChem Substance'] = reactome_to_pubchem[reactome_id]
        
        # 将结果保存到新的Excel文件
        output_file = os.path.splitext(input_file)[0] + "_with_pubchem.xlsx"
        df.to_excel(output_file, index=False)
        print(f"\n已将更新数据保存到 {output_file}")
        print(f"成功获取 {success_count} 个唯一PubChem Substance ID (共处理 {processed_count} 个唯一Reactome ID)")
        print(f"总共更新了 {df['PubChem Substance'].notna().sum()} 行数据")
        
    finally:
        # 确保关闭浏览器
        print("正在关闭浏览器...")
        driver.quit()
        
    return df

if __name__ == "__main__":
    # 使用指定的输入文件
    input_file = 'met_updated.xlsx'
    print(f"使用指定的输入文件: {input_file}")
    result_df = scrape_reactome_pubchem(input_file)

使用指定的输入文件: met_updated.xlsx
正在读取Excel文件: met_updated.xlsx
总行数: 1652
包含Reactome Compound ID的行数: 509
唯一Reactome Compound ID数量: 383

正在初始化Edge WebDriver...

正在处理唯一ID 1/383: R-ALL-113599
访问URL: https://reactome.org/content/detail/R-ALL-113599#Homo%20sapiens
找到 1 个PubChem Substance标签
方法1找到PubChem ID: 8144165

正在处理唯一ID 2/383: R-ALL-2162265
访问URL: https://reactome.org/content/detail/R-ALL-2162265#Homo%20sapiens
找到 1 个PubChem Substance标签
方法1找到PubChem ID: 8145813

正在处理唯一ID 3/383: R-ALL-5359061
访问URL: https://reactome.org/content/detail/R-ALL-5359061#Homo%20sapiens
未找到PubChem Substance标签
未找到任何PubChem链接
未在Cross References部分找到PubChem信息

正在处理唯一ID 4/383: R-ALL-112032
访问URL: https://reactome.org/content/detail/R-ALL-112032#Homo%20sapiens
未找到PubChem Substance标签
未找到任何PubChem链接
未在Cross References部分找到PubChem信息

正在处理唯一ID 5/383: R-ALL-29666
访问URL: https://reactome.org/content/detail/R-ALL-29666#Homo%20sapiens
未找到PubChem Substance标签
未找到任何PubChem链接
未在Cross References部分找到PubChem信息

正在处理唯一ID 6/383: R-ALL-16268

# 完善smiles

In [ ]:
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def process_excel(file_path):
    # 读取Excel文件
    print("正在读取Excel文件...")
    df = pd.read_excel(file_path, sheet_name='metabolites')
    print(f"Excel文件读取成功，共有{len(df)}行数据")

    # 以更安全的方式筛选数据
    # 单独计算每个条件，然后合并
    condition1 = df['smiles'].isna()
    # 对于非NaN值，检查是否为空字符串
    condition2 = df['smiles'].notna() & (df['smiles'].astype(str).str.strip() == '')
    # 合并成一个条件：smiles为空
    empty_smiles = condition1 | condition2

    # 检查metabolites_id是否符合格式
    cpd_pattern = r'cpd\d+'
    has_cpd_format = df['metabolites_id'].astype(str).str.contains(cpd_pattern, regex=True)

    # 合并所有条件
    filtered_df = df[empty_smiles & has_cpd_format]

    print(f"空smiles的行数: {empty_smiles.sum()}")
    print(f"符合cpd格式的行数: {has_cpd_format.sum()}")
    print(f"同时符合两个条件的行数: {len(filtered_df)}")

    # 如果没有符合条件的行，返回
    if len(filtered_df) == 0:
        print("没有找到符合条件的行，程序将退出")
        return {}

    # 显示筛选出的行
    print("\n筛选出的行:")
    for i, (idx, row) in enumerate(filtered_df.head(5).iterrows()):
        print(f"  {i+1}. {row['metabolites_id']} - {row['name']}")

    if len(filtered_df) > 5:
        print(f"  ...以及 {len(filtered_df)-5} 条更多记录")

    # 创建字典存储名称和SMILES对
    results = {}

    # 设置Edge浏览器驱动
    print("\n正在初始化Edge WebDriver...")
    edge_options = Options()
    edge_options.add_argument("--headless")  # 如果你不希望打开浏览器窗口，取消注释此行
    driver = webdriver.Edge(options=edge_options)

    try:
        # 对每个名称，从PubChem获取SMILES
        for index, row in filtered_df.iterrows():
            name = row['name']
            metabolite_id = row['metabolites_id']

            print(f"\n正在处理 {metabolite_id}: {name}")

            try:
                # 构建URL
                search_url = f"https://pubchem.ncbi.nlm.nih.gov/#query={name}"
                print(f"访问URL: {search_url}")
                driver.get(search_url)

                # 等待搜索结果加载
                wait = WebDriverWait(driver, 15)
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "f-medium")))
                print("搜索结果已加载")

                # 等待页面加载完成
                time.sleep(3)  # 给页面一些时间完全加载

                # 尝试多种定位方法
                smiles = None

                # 方法1：使用你提供的XPath
                try:
                    precise_xpath = "/html/body/div[1]/div/div/main/div[2]/div[1]/div/div[2]/div/div[1]/div[2]/div[5]/div/span/span[2]/span"
                    smiles_element = driver.find_element(By.XPATH, precise_xpath)
                    smiles = smiles_element.text
                    print(f"方法1成功: {smiles}")
                except Exception as e:
                    print(f"方法1失败: {e}")


                # 如果找到了SMILES数据，保存结果
                if smiles:
                    results[metabolite_id] = {'name': name, 'smiles': smiles}
                    print(f"找到 {name} 的SMILES: {smiles}")
                else:
                    print(f"无法找到 {name} 的SMILES")


            except Exception as e:
                print(f"处理 {name} 时出错: {e}")

            # 暂停以避免过度请求服务器
            time.sleep(2)

        # 使用新的SMILES值更新原始数据框
        if results:
            for metabolite_id, data in results.items():
                df.loc[df['metabolites_id'] == metabolite_id, 'smiles'] = data['smiles']

            # 将更新后的数据框保存到新的Excel文件
            output_file = "smiles.xlsx"
            df.to_excel(output_file, index=False)
            print(f"\n已将更新后的数据保存到 {output_file}")

    finally:
        # 确保无论如何都关闭浏览器
        print("正在关闭浏览器...")
        driver.quit()

    return results

if __name__ == "__main__":
    # 替换为你的实际文件路径
    file_path = "model_history/14067gem19.xlsx"
    results = process_excel(file_path)

    # 显示结果摘要
    print("\n结果摘要:")
    for metabolite_id, data in results.items():
        print(f"{metabolite_id} ({data['name']}): {data['smiles']}")

    # 关闭浏览器
    driver.quit()

# 完善反应的definition

In [1]:
import pandas as pd
import re
import os

def process_excel(file_path, output_path=None):
    """
    处理Excel文件，替换reactions工作簿中的Equation列的化合物ID为名称，
    并将结果保存在Definition列中
    """
    if output_path is None:
        # 如果没有提供输出路径，创建在同一目录下的新文件
        dir_name, file_name = os.path.split(file_path)
        name, ext = os.path.splitext(file_name)
        output_path = os.path.join(dir_name, f"{name}_processed{ext}")
    
    print(f"正在读取Excel文件: {file_path}")
    
    # 读取Excel文件的两个工作簿
    try:
        metabolites_df = pd.read_excel(file_path, sheet_name='metabolites')
        reactions_df = pd.read_excel(file_path, sheet_name='reactions')
        print(f"成功读取metabolites工作簿，共{len(metabolites_df)}行")
        print(f"成功读取reactions工作簿，共{len(reactions_df)}行")
    except Exception as e:
        print(f"读取Excel工作簿时出错: {e}")
        return
    
    # 创建映射字典 - 从metabolites_id中提取基础ID（不带后缀）
    metabolite_map = {}
    for _, row in metabolites_df.iterrows():
        met_id = row['metabolites_id']
        name = row['name']
        
        # 分离基础ID和后缀
        if '_' in met_id:
            base_id, _ = met_id.split('_', 1)
            metabolite_map[base_id] = name
        else:
            metabolite_map[met_id] = name
    
    print(f"成功创建映射，共{len(metabolite_map)}个代谢物")
    
    # 定义替换函数
    def replace_compounds(equation):
        if pd.isna(equation):
            return equation
        
        # 正则表达式：匹配(数字) 后跟代谢物ID和后缀
        pattern = r'\(([0-9.]+)\) (cpd\d+)(_[ce])'
        
        def replacer(match):
            coef = match.group(1)       # 系数
            base_id = match.group(2)    # 基础ID（不带后缀）
            suffix = match.group(3)     # 后缀（_c或_e）
            
            # 查找对应的名称并保留后缀
            if base_id in metabolite_map:
                return f'({coef}) {metabolite_map[base_id]}{suffix}'
            else:
                # 如果找不到映射，保持原样
                return match.group(0)
        
        # 替换方程式中的所有化合物ID
        result = re.sub(pattern, replacer, equation)
        
        # 特殊处理"protein_c"
        if 'protein_c' in result:
            protein_pattern = r'\(([0-9.]+)\) protein(_c)'
            
            def protein_replacer(match):
                coef = match.group(1)
                suffix = match.group(2)
                if 'protein' in metabolite_map:
                    return f'({coef}) {metabolite_map["protein"]}{suffix}'
                else:
                    return match.group(0)
            
            result = re.sub(protein_pattern, protein_replacer, result)
        
        return result
    
    # 应用替换函数到Equation列并保存到Definition列
    print("开始处理方程式...")
    if 'Definition' not in reactions_df.columns:
        reactions_df['Definition'] = ''  # 如果Definition列不存在，创建它
    
    reactions_df['Definition'] = reactions_df['Equation'].apply(replace_compounds)
    
    # 保存前展示部分结果
    print("\n处理结果示例:")
    for i in range(min(3, len(reactions_df))):
        eq = reactions_df.iloc[i]['Equation']
        if pd.notna(eq) and 'cpd' in eq:
            print(f"原始方程: {eq}")
            print(f"替换后: {reactions_df.iloc[i]['Definition']}")
            print()
    
    # 保存结果到Excel文件
    print(f"保存结果到: {output_path}")
    try:
        # 读取原始Excel文件的所有工作簿
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # 复制所有工作簿
            xlsx = pd.ExcelFile(file_path)
            for sheet_name in xlsx.sheet_names:
                if sheet_name == 'reactions':
                    # 保存修改后的reactions工作簿
                    reactions_df.to_excel(writer, sheet_name=sheet_name, index=False)
                else:
                    # 复制其他工作簿
                    pd.read_excel(xlsx, sheet_name=sheet_name).to_excel(writer, sheet_name=sheet_name, index=False)
        
        print("处理完成，结果已保存到Excel文件")
    except Exception as e:
        print(f"保存结果时出错: {e}")

# 使用示例
if __name__ == "__main__":
    file_path = "14067gem20_CGXII.xlsx"  # 您的Excel文件路径
    process_excel(file_path)

正在读取Excel文件: 14067gem20_CGXII.xlsx
成功读取metabolites工作簿，共1622行
成功读取reactions工作簿，共1723行
成功创建映射，共1417个代谢物
开始处理方程式...

处理结果示例:
原始方程: (1) cpd11427_c + (1) cpd00080_c => (1) cpd11652_c + (1) cpd00046_c
替换后: (1) CDP-diacylglycerol_c + (1) Glycerol-3-phosphate_c => (1) Phosphatidylglycerol_c + (1) CMP_c

原始方程: (4.65) cpd00691_c + (0.498) cpd00043_c => (1) arabino_c
替换后: (4.65) UDP-L-arabinose_c + (0.498) UDP-galactose_c => (1) arabino_c

原始方程: (0.095) cpd02943_c + (0.095) arabino_c => (1) Cellwall_c
替换后: (0.095) UDP-N-acetylmuramoyl-L-alanyl-D-glutamyl-L-lysyl-D-alanyl-D-alanine_c + (0.095) arabino_c => (1) Cellwall_c

保存结果到: 14067gem20_CGXII_processed.xlsx
处理完成，结果已保存到Excel文件
